In [3]:
import numpy as np

#main LU decomposition logic
def getL(LU):
    dim = LU.shape[0]
    l = LU.copy()
    for k in range(dim):
            l[k, k] = 1
            l[k, k + 1:] = 0
    return l

def getU(LU):
    dim = LU.shape[0]
    u = LU.copy()
    for k in range(1, dim):
        u[k, :k] = 0
    return u

def getLU(a):
    dim = a.shape[0]
    a = a.copy()
    detSgn = 1
    p = np.matrix(np.zeros(shape=(dim, dim)))
    q = np.matrix(np.zeros(shape=(dim, dim)))
    np.fill_diagonal(p, 1)
    np.fill_diagonal(q, 1)
    for k in range(dim - 1): 
        (choseRow, choseCol) = np.unravel_index(np.argmax(np.abs(a[k:, k:])), a[k:, k:].shape)
        choseRow += k
        choseCol += k
        if choseRow != k:
            detSgn *= -1
            a[[choseRow, k]] = a[[k, choseRow]]
            p[:, [choseRow, k]] = p[:, [k, choseRow]]    
        if choseCol != k:
            detSgn *= -1
            a[:, [choseCol, k]] = a[:, [k, choseCol]]
            q[[choseCol, k]] = q[[k, choseCol]]
        if np.abs(a[k, k]) < pres:
            return p, getL(a), getU(a), q, detSgn
        for j in range(k + 1, dim):
            coef = a[j, k] / a[k, k]
            a[j, k:] = a[j, k:] - a[k, k:] * coef
            a[j, k] = coef
    return p, getL(a), getU(a), q, detSgn

#determinant calc logic
def getDet(a):
    dim = a.shape[0]
    p, l, u, q, Det = getLU(a)
    for k in range(dim):
        Det *= u[k, k]
    return Det

#solving
def directSub(l, B, p):
    dim = p.shape[0]
    B = p.transpose() * B
    Y = np.matrix(np.zeros(shape=(dim, 1)))
    for k in range(dim):
        summ = 0
        for j in range(k):
            summ += l[k, j] * Y[j, 0]
        Y[k, 0] = B[k, 0] - summ
    return Y

def inverceSub(u, Y, p, q):
    dim = p.shape[0]
    x_ = np.matrix(np.zeros(shape=(dim, 1)))
    for k in range(dim):
        summ = 0
        for j in range(k):
            summ += u[dim - 1 - k, dim - 1 - j] * x_[dim - 1 - j, 0]
        x_[dim - 1 - k, 0] = (Y[dim - 1 - k, 0] - summ) / u[dim - 1 - k, dim - 1 - k]
    return (x_.transpose() * q).transpose()

def solve(a, b_):
    rangA = getRang(a)
    if rangA == a.shape[0]:
        p, l, u, q, Det = getLU(a)
        Y = directSub(l, b_, p)
        x_ = inverceSub(u, Y, p, q)
        return x_
    if getComp(a, b_):
        p, l, u, q, Det = getLU(a)
        for k in range(rangA, n):
            u[k, k] = 1.
        Y = directSub(l, b_, p)
        x_ = inverceSub(u, Y, p, q)
        return x_
    else:
        return 0

#reverce matrix calc
def getInvMatrix(a):
    dim = a.shape[0]
    revMatr = np.matrix(np.zeros(shape=(dim, dim)))
    for k in range(dim):
        Ima = np.matrix(np.zeros(shape=(dim, 1)))
        Ima[k, 0] = 1
        revMatr[k] = solve(a, Ima).transpose()
    return revMatr.transpose()

#condition number
def getNorm(a):
    dim = a.shape[0]
    Cols = []
    for k in range(dim):
        curColSumm = 0
        for j in range(dim):
            curColSumm += abs(a[k, j])
        Cols.append(curColSumm)
    return max(Cols)

def getCondition(a):
    Arev = getInvMatrix(a)
    normA = getNorm(a)
    normRevA = getNorm(Arev)
    return normA*normRevA

#rang & compatibility
def getRang(a):
    dim = a.shape[0]
    p, l, u, q, Det = getLU(a)
    rang = 0
    for k in range(dim):
        if np.abs(u[k, dim - 1]) < pres:
            return rang
        rang += 1
    return rang

def getComp(a, b_):
    dim = a.shape[0]
    rangA = getRang(a)
    AB = np.concatenate((a, b_), axis=1)
    np.matrix.resize(AB, (dim + 1, dim + 1))
    rangAB = getRang(AB)
    if rangA == rangAB:
        return True
    else:
        return False

In [4]:
#create random n*n matrix A
pres = 0.1**12
n = np.random.randint(3, 5)
A = np.matrix(np.random.randn(n, n))*n*100
print("matrix A: ")
print(A)

#test 0
P, L, U, Q, detSight = getLU(A)
print("\n", "L: ")
print(L)
print("\n", "U: ")
print(U)
print("\n", "A - P*L*U*Q: ")
print(A - P*L*U*Q)

#test 1 (determinant)
print("\n", "np.linalg.det(A) & LU Det(A): ")
print(np.linalg.det(A), getDet(A))

#test 2 (solving)
b = np.matrix(np.random.randn(n, 1))*n*100
print("\n", "A*x - b: ")
x = solve(A, b)
print(b - A*x)

#test 3 (reverce matrix)
revA = getInvMatrix(A)
print("\n", "A*A^-1: ")
print(A*revA)

#test 4 (condition number)
print("\n", "np.linalg.cond(A, np.inf) & getCondition(A): ")
print(np.linalg.cond(A, np.inf), getCondition(A))

#test 5 (singular matrix)
SingA = A.copy()
SingB = b.copy()
rand = np.random.randint(2, size=n)
for i in range(n):
    if rand[i] == 1:
        SingA[i] = SingA[0]*n
        #SingB[i] = SingB[0]*n
P, L, U, Q, detSight = getLU(SingA)
print("\n", "singular A: ")
print(SingA)
print("\n", "L: ")
print(L)
print("\n", "U: ")
print(U)
print("\n", "np.linalg.matrix_rank(SingA) & getRang(SingA): ")
print(np.linalg.matrix_rank(SingA), getRang(SingA))
print("\n", "is compatible system:")
if getComp(SingA, SingB):
    print(True)
    X = solve(SingA, SingB)
    print("\n", "SingA*X - SingB: ")
    print(SingB - SingA*X)
else:
    print(False)

matrix A: 
[[ -31.8061291   368.76551564  405.96474155 -372.38235789]
 [ -87.38689511  598.45558692 -658.35728721 -116.54919295]
 [ 210.75673678   36.40398658 -278.68828425 -994.65919704]
 [ 445.79045963  167.42776779  230.32969785  546.15375733]]

 L: 
[[ 1.          0.          0.          0.        ]
 [ 0.117175    1.          0.          0.        ]
 [ 0.37438186 -0.8155649   1.          0.        ]
 [-0.54908632 -0.12355047  0.31060814  1.        ]]

 U: 
[[-994.65919704 -278.68828425   36.40398658  210.75673678]
 [   0.         -625.70198707  594.18994973 -112.08231607]
 [   0.            0.          839.73699102 -202.12003072]
 [   0.            0.            0.          610.44640474]]

 A - P*L*U*Q: 
[[-1.42108547e-14 -5.68434189e-14  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.13686838e-13  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 np.linalg.d